<a href="https://github.com/timeseriesAI/tsai-rs" target="_parent"><img src="https://img.shields.io/badge/tsai--rs-Time%20Series%20AI%20in%20Rust-blue" alt="tsai-rs"/></a>

# Time Series Transforms with tsai-rs

This notebook demonstrates time series data transforms using **tsai-rs**, a Rust implementation with Python bindings.

## Purpose

Time series transforms are essential for:
1. Data preprocessing (standardization, normalization)
2. Data augmentation (noise, scaling, warping)
3. Time series to image conversion (GASF, GADF, Recurrence Plots)

tsai-rs provides efficient Rust implementations of these transforms with Python bindings.

## Install tsai-rs

```bash
cd crates/tsai_python
maturin develop --release
```

## Import Libraries

In [ ]:
import tsai_rs
import numpy as np
import matplotlib.pyplot as plt

print(f"tsai-rs version: {tsai_rs.version()}")
tsai_rs.my_setup()

## Load Sample Data

In [ ]:
# Load a multivariate dataset
dsid = 'NATOPS'
X_train, y_train, X_test, y_test = tsai_rs.get_UCR_data(dsid, return_split=True)

print(f"Dataset: {dsid}")
print(f"X_train shape: {X_train.shape} (samples, variables, length)")
print(f"X_test shape: {X_test.shape}")

## Standardization Transforms

Standardization is crucial for neural network training. tsai-rs supports multiple standardization modes.

In [ ]:
# Convert to float32 for transforms
X = X_train.astype(np.float32)

print(f"Original data statistics:")
print(f"  Mean: {X.mean():.4f}")
print(f"  Std: {X.std():.4f}")
print(f"  Min: {X.min():.4f}")
print(f"  Max: {X.max():.4f}")

In [ ]:
# Standardize by sample (each sample has mean=0, std=1)
X_std_sample = tsai_rs.ts_standardize(X, by_sample=True)

print(f"\nStandardized by sample:")
print(f"  Sample 0 mean: {X_std_sample[0].mean():.6f}")
print(f"  Sample 0 std: {X_std_sample[0].std():.6f}")

In [ ]:
# Standardize globally (entire dataset has mean=0, std=1)
X_std_global = tsai_rs.ts_standardize(X, by_sample=False)

print(f"\nStandardized globally:")
print(f"  Dataset mean: {X_std_global.mean():.6f}")
print(f"  Dataset std: {X_std_global.std():.6f}")

## Data Augmentation Transforms

Data augmentation helps improve model generalization by creating variations of the training data.

### Gaussian Noise

In [ ]:
# Add Gaussian noise to time series
X_noisy = tsai_rs.add_gaussian_noise(X_std_sample, std=0.1, seed=42)

# Visualize original vs noisy
fig, axes = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

sample_idx = 0
var_idx = 0

axes[0].plot(X_std_sample[sample_idx, var_idx, :], label='Original')
axes[0].set_title('Original Signal')
axes[0].legend()

axes[1].plot(X_noisy[sample_idx, var_idx, :], label='With Gaussian Noise', color='orange')
axes[1].set_title('Signal with Gaussian Noise (std=0.1)')
axes[1].legend()

plt.tight_layout()
plt.show()

In [ ]:
# Different noise levels
noise_levels = [0.05, 0.1, 0.2, 0.5]

fig, axes = plt.subplots(len(noise_levels) + 1, 1, figsize=(12, 10), sharex=True)

axes[0].plot(X_std_sample[0, 0, :])
axes[0].set_title('Original')

for i, std in enumerate(noise_levels):
    X_aug = tsai_rs.add_gaussian_noise(X_std_sample, std=std, seed=42)
    axes[i+1].plot(X_aug[0, 0, :])
    axes[i+1].set_title(f'Gaussian Noise (std={std})')

plt.tight_layout()
plt.show()

### Magnitude Scaling

In [ ]:
# Magnitude scaling - randomly scales the amplitude
X_scaled = tsai_rs.mag_scale(X_std_sample, scale_range=(0.8, 1.2), seed=42)

fig, axes = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

axes[0].plot(X_std_sample[0, 0, :], label='Original')
axes[0].set_title('Original Signal')
axes[0].legend()

axes[1].plot(X_scaled[0, 0, :], label='Magnitude Scaled', color='green')
axes[1].set_title('Magnitude Scaled (0.8-1.2x)')
axes[1].legend()

plt.tight_layout()
plt.show()

print(f"Original max: {X_std_sample[0, 0, :].max():.4f}")
print(f"Scaled max: {X_scaled[0, 0, :].max():.4f}")

### Combining Transforms

In [ ]:
# Apply multiple transforms in sequence
X_aug = X_std_sample.copy()
X_aug = tsai_rs.mag_scale(X_aug, scale_range=(0.9, 1.1), seed=42)
X_aug = tsai_rs.add_gaussian_noise(X_aug, std=0.05, seed=43)

fig, axes = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

axes[0].plot(X_std_sample[0, 0, :], label='Original')
axes[0].set_title('Original Signal')

axes[1].plot(X_aug[0, 0, :], label='Augmented', color='purple')
axes[1].set_title('Augmented (Scale + Noise)')

plt.tight_layout()
plt.show()

## Time Series to Image Transforms

These transforms convert 1D time series into 2D images, enabling the use of image-based deep learning models.

In [ ]:
# Get a single univariate time series for demonstration
sample_ts = X_std_sample[0, 0, :].astype(np.float32)
print(f"Time series shape: {sample_ts.shape}")
print(f"Time series range: [{sample_ts.min():.4f}, {sample_ts.max():.4f}]")

### GASF (Gramian Angular Summation Field)

In [ ]:
# Compute GASF image
gasf_image = tsai_rs.compute_gasf(sample_ts)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(sample_ts)
axes[0].set_title('Original Time Series')
axes[0].set_xlabel('Time')

im = axes[1].imshow(gasf_image, cmap='viridis', aspect='auto')
axes[1].set_title(f'GASF Image ({gasf_image.shape[0]}x{gasf_image.shape[1]})')
plt.colorbar(im, ax=axes[1])

plt.tight_layout()
plt.show()

### GADF (Gramian Angular Difference Field)

In [ ]:
# Compute GADF image
gadf_image = tsai_rs.compute_gadf(sample_ts)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(sample_ts)
axes[0].set_title('Original Time Series')
axes[0].set_xlabel('Time')

im = axes[1].imshow(gadf_image, cmap='plasma', aspect='auto')
axes[1].set_title(f'GADF Image ({gadf_image.shape[0]}x{gadf_image.shape[1]})')
plt.colorbar(im, ax=axes[1])

plt.tight_layout()
plt.show()

### Recurrence Plot

In [ ]:
# Compute Recurrence Plot
rp_image = tsai_rs.compute_recurrence_plot(sample_ts, threshold=0.1)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(sample_ts)
axes[0].set_title('Original Time Series')
axes[0].set_xlabel('Time')

im = axes[1].imshow(rp_image, cmap='binary', aspect='auto')
axes[1].set_title(f'Recurrence Plot ({rp_image.shape[0]}x{rp_image.shape[1]})')
plt.colorbar(im, ax=axes[1])

plt.tight_layout()
plt.show()

In [ ]:
# Compare different threshold values for Recurrence Plot
thresholds = [0.05, 0.1, 0.2, 0.5]

fig, axes = plt.subplots(1, len(thresholds), figsize=(16, 4))

for i, thresh in enumerate(thresholds):
    rp = tsai_rs.compute_recurrence_plot(sample_ts, threshold=thresh)
    axes[i].imshow(rp, cmap='binary', aspect='auto')
    axes[i].set_title(f'Threshold={thresh}')

plt.suptitle('Recurrence Plots with Different Thresholds')
plt.tight_layout()
plt.show()

### Comparing All TS-to-Image Transforms

In [ ]:
# Compare all transforms side by side
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Original time series
axes[0, 0].plot(sample_ts)
axes[0, 0].set_title('Original Time Series')

# GASF
gasf = tsai_rs.compute_gasf(sample_ts)
im1 = axes[0, 1].imshow(gasf, cmap='viridis', aspect='auto')
axes[0, 1].set_title('GASF')
plt.colorbar(im1, ax=axes[0, 1])

# GADF
gadf = tsai_rs.compute_gadf(sample_ts)
im2 = axes[1, 0].imshow(gadf, cmap='plasma', aspect='auto')
axes[1, 0].set_title('GADF')
plt.colorbar(im2, ax=axes[1, 0])

# Recurrence Plot
rp = tsai_rs.compute_recurrence_plot(sample_ts, threshold=0.1)
im3 = axes[1, 1].imshow(rp, cmap='binary', aspect='auto')
axes[1, 1].set_title('Recurrence Plot')
plt.colorbar(im3, ax=axes[1, 1])

plt.tight_layout()
plt.show()

## Applying Transforms to Different Classes

In [ ]:
# Get samples from different classes
unique_classes = np.unique(y_train)
print(f"Classes: {unique_classes}")

# Create GASF images for one sample from each class
fig, axes = plt.subplots(2, len(unique_classes), figsize=(18, 8))

for i, cls in enumerate(unique_classes):
    # Get first sample of this class
    idx = np.where(y_train == cls)[0][0]
    ts = X_std_sample[idx, 0, :].astype(np.float32)
    
    # Plot time series
    axes[0, i].plot(ts)
    axes[0, i].set_title(f'Class {cls}')
    
    # Plot GASF
    gasf = tsai_rs.compute_gasf(ts)
    axes[1, i].imshow(gasf, cmap='viridis', aspect='auto')
    axes[1, i].set_title(f'GASF - Class {cls}')

axes[0, 0].set_ylabel('Time Series')
axes[1, 0].set_ylabel('GASF Image')

plt.tight_layout()
plt.show()

## Transform Pipeline Example

In [ ]:
def augment_and_transform(X, n_augmented=5, seed=42):
    """
    Create augmented versions and compute GASF images.
    
    Args:
        X: Input data (samples, vars, length)
        n_augmented: Number of augmented versions per sample
        seed: Random seed
    
    Returns:
        List of GASF images
    """
    # Standardize
    X_std = tsai_rs.ts_standardize(X.astype(np.float32), by_sample=True)
    
    all_gasf = []
    
    for i in range(n_augmented):
        # Apply augmentation with different seeds
        X_aug = tsai_rs.mag_scale(X_std, scale_range=(0.9, 1.1), seed=seed + i)
        X_aug = tsai_rs.add_gaussian_noise(X_aug, std=0.05, seed=seed + i + 100)
        
        # Compute GASF for first sample, first variable
        gasf = tsai_rs.compute_gasf(X_aug[0, 0, :])
        all_gasf.append(gasf)
    
    return all_gasf

# Generate augmented GASF images
gasf_images = augment_and_transform(X_train[:1], n_augmented=5)

# Visualize
fig, axes = plt.subplots(1, 5, figsize=(15, 3))
for i, gasf in enumerate(gasf_images):
    axes[i].imshow(gasf, cmap='viridis', aspect='auto')
    axes[i].set_title(f'Augmented {i+1}')
    axes[i].axis('off')

plt.suptitle('GASF Images from Augmented Time Series')
plt.tight_layout()
plt.show()

## Summary

This notebook demonstrated the key transforms available in tsai-rs:

### Preprocessing Transforms
- `ts_standardize`: Standardize time series (by sample or globally)

### Augmentation Transforms  
- `add_gaussian_noise`: Add random Gaussian noise
- `mag_scale`: Random magnitude scaling

### Time Series to Image Transforms
- `compute_gasf`: Gramian Angular Summation Field
- `compute_gadf`: Gramian Angular Difference Field  
- `compute_recurrence_plot`: Recurrence Plot

These transforms can be combined in pipelines to preprocess data and create augmented training sets for improved model performance.

In [ ]:
# Quick reference example
dsid = 'NATOPS'
X_train, y_train, X_test, y_test = tsai_rs.get_UCR_data(dsid, return_split=True)

# Preprocess pipeline
X = X_train.astype(np.float32)
X_std = tsai_rs.ts_standardize(X, by_sample=True)
X_aug = tsai_rs.add_gaussian_noise(X_std, std=0.05, seed=42)
X_aug = tsai_rs.mag_scale(X_aug, scale_range=(0.9, 1.1), seed=42)

# Generate image representation
sample_ts = X_aug[0, 0, :]
gasf = tsai_rs.compute_gasf(sample_ts)
gadf = tsai_rs.compute_gadf(sample_ts)
rp = tsai_rs.compute_recurrence_plot(sample_ts, threshold=0.1)

print(f"Preprocessed data shape: {X_aug.shape}")
print(f"GASF image shape: {gasf.shape}")
print(f"GADF image shape: {gadf.shape}")
print(f"Recurrence plot shape: {rp.shape}")